In [2]:
import tensorflow as tf
from keras.layers import LSTM, GRU, Dense, Embedding, Dropout , Conv1D , GlobalMaxPool1D
from keras.preprocessing import text, sequence
from keras.models import Sequential
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from keras.utils.data_utils import pad_sequences
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from keras.models import load_model
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import pandas as pd 
from sklearn import preprocessing
import re
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.linear_model import LogisticRegression
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import pickle

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [3]:
tokenizer = RegexpTokenizer(r'\w+')
 
nltk.download('stopwords')
stopwords_list = stopwords.words('english')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ALI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
kdnugget_dataset = pd.read_csv("fake_and_real_news_dataset.csv")
kdnugget_dataset["Label"] = kdnugget_dataset.apply(lambda x: 1 if x["label"] == "REAL" else 0  , axis=1) 
kdnugget_dataset = kdnugget_dataset.sample(frac=1).reset_index(drop=True)
df = kdnugget_dataset.copy()


In [14]:
kdnugget_dataset.head()

,idd,title,text,label,Label
0,McoQ#gZlJT,Ted Cruz’s Judgment Day,"HOUSTON — It was July, only four months into t...",REAL,1
1,iImyo10apD,"Anti-Trump Protest In Portland, Ore., Turns De...","A Portland, Ore., demonstration against Presid...",REAL,1
2,m1G&_SqMDC,Rubio's secret weapon,A verdict in 2017 could have sweeping conseque...,REAL,1
3,NpMPPJ853X,How Will Clinton – Comey Affect Gold Prices?,Home » Gold » Gold News » How Will Clinton – C...,FAKE,0
4,DDSbr5nAJJ,Malcolm X on Hillary Clinton and Donald Trump,A Black Agenda Radio Commentary by Bruce A. ...,FAKE,0


In [15]:
kdnugget_dataset["Title with text"] = kdnugget_dataset.apply(lambda x: x["title"] +  x["text"] if type(x["title"]) != float else x["text"]  , axis=1) 
kdnugget_dataset = kdnugget_dataset.to_numpy()
kdnugget_y = kdnugget_dataset[:, 4]
kdnugget_X = kdnugget_dataset[:, 5]
kdnugget_X_postp = []
for i in range(kdnugget_X.shape[0]):
    sentence = kdnugget_X[i]
    sentence = re.sub(r'[^a-zA-Z ]', '', sentence)
    for stop_word in stopwords_list:
        sentence = sentence.replace(" " + stop_word + " ", " ")

    kdnugget_X_postp.append(sentence)
kdnugget_X = kdnugget_X_postp 

In [16]:
vectorizer = TfidfVectorizer()
kdnugget_X_tfidf = vectorizer.fit_transform(kdnugget_X)
vectorizer = TfidfVectorizer(use_idf=False)
kdnugget_X_tf = vectorizer.fit_transform(kdnugget_X)

In [19]:
df.shape
df.head()

,idd,title,text,label,Label
0,McoQ#gZlJT,Ted Cruz’s Judgment Day,"HOUSTON — It was July, only four months into t...",REAL,1
1,iImyo10apD,"Anti-Trump Protest In Portland, Ore., Turns De...","A Portland, Ore., demonstration against Presid...",REAL,1
2,m1G&_SqMDC,Rubio's secret weapon,A verdict in 2017 could have sweeping conseque...,REAL,1
3,NpMPPJ853X,How Will Clinton – Comey Affect Gold Prices?,Home » Gold » Gold News » How Will Clinton – C...,FAKE,0
4,DDSbr5nAJJ,Malcolm X on Hillary Clinton and Donald Trump,A Black Agenda Radio Commentary by Bruce A. ...,FAKE,0


In [20]:
def clean_text(txt):

    contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}
    def _get_contractions(contraction_dict):
        contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
        return contraction_dict, contraction_re

    def replace_contractions(text):
        contractions, contractions_re = _get_contractions(contraction_dict)
        def replace(match):
            return contractions[match.group(0)]
        return contractions_re.sub(replace, text)

    # replace contractions
    txt = replace_contractions(txt)
    
    #remove punctuations
    txt  = "".join([char for char in txt if char not in string.punctuation])
    txt = re.sub('[0-9]+', '', txt)
    
    # split into words
    words = word_tokenize(txt)
    
    # remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    
    # removing leftover punctuations
    words = [word for word in words if word.isalpha()]
    
    cleaned_text = ' '.join(words)
    return cleaned_text

df['data_cleaned'] = df['title'].apply(lambda txt: clean_text(txt) if type(txt) != float else str(txt))


In [21]:
xtrain, xtest, ytrain, ytest = train_test_split(df['data_cleaned'], df['Label'], test_size=0.2 , shuffle=False)
kdnugget_X_train, kdnugget_X_test, kdnugget_y_train, kdnugget_y_test = train_test_split(kdnugget_X_tfidf, kdnugget_y, test_size=0.2 , shuffle=False)
kdnugget_X_train_tf, kdnugget_X_test_tf, kdnugget_y_train_tf, kdnugget_y_test_tf = train_test_split(kdnugget_X_tf, kdnugget_y, test_size=0.2 , shuffle=False)


In [22]:
# find the length of the largest sentence in training data
max_len = xtrain.apply(lambda x: len(x)).max()


In [23]:
max_words = 10000
tokenizer = text.Tokenizer(num_words = max_words)
# create the vocabulary by fitting on x_train text
tokenizer.fit_on_texts(xtrain)
# generate the sequence of tokens
xtrain_seq = tokenizer.texts_to_sequences(xtrain)
xtest_seq = tokenizer.texts_to_sequences(xtest)

# pad the sequences
xtrain_pad = pad_sequences(xtrain_seq, maxlen=max_len)
xtest_pad = pad_sequences(xtest_seq, maxlen=max_len)
word_index = tokenizer.word_index

print('Text Example:', xtrain[0])
print('Sequence of indices(before padding):', xtrain_seq[0])
print('Sequence of indices(after padding):', xtrain_pad[0])

Text Example: Ted Cruz Judgment Day
Sequence of indices(before padding): [86, 25, 1192, 81]
Sequence of indices(after padding): [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0

In [24]:
def eval_model(true_value, predict_value):
    print("accuracy:  ", accuracy_score(true_value, predict_value))
    print("Precision:  ", precision_score(true_value, predict_value))
    print("Recall:  ", recall_score(true_value, predict_value))
    print("F1 score:  ", f1_score(true_value, predict_value))


In [25]:
log_model = pickle.load(open("log_model_kdnugget.pickle", "rb"))
predict_y_lr = log_model.predict(kdnugget_X_train)


In [26]:
svm_clf = pickle.load(open("svm_clf_kdnugget.pickle", "rb"))
predict_y_svm = svm_clf.predict(kdnugget_X_train)


In [27]:
dt_clf = pickle.load(open("dt_clf_kdnugget.pickle", "rb"))
predict_y_dt = dt_clf.predict(kdnugget_X_train)


In [28]:
knn = pickle.load(open("knn_kdnugget.pickle", "rb"))
predict_y_knn = knn.predict(kdnugget_X_train)


In [29]:
rf1_clf = pickle.load(open("rf1_clf_kdnugget.pickle", "rb"))
predict_y_rf1 = rf1_clf.predict(kdnugget_X_train)


In [30]:
rf2_clf = pickle.load(open("rf2_clf_kdnugget.pickle", "rb"))
predict_y_rf2 = rf2_clf.predict(kdnugget_X_train_tf)


In [31]:
lstm_model = load_model('lstm_model_kdnugget.h5')
predict_y_lstm = lstm_model.predict(xtrain_pad).round().astype(int).reshape(-1)


115/115 [==============================] - 7s 58ms/step


In [32]:
gru_model = load_model('gru_model_kdnugget.h5')
predict_y_gru = gru_model.predict(xtrain_pad).round().astype(int).reshape(-1)


115/115 [==============================] - 3s 26ms/step


In [33]:
cnn_model = load_model('cnn_model_kdnugget.h5')
predict_y_cnn = cnn_model.predict(xtrain_pad).round().astype(int).reshape(-1)


115/115 [==============================] - 1s 5ms/step


In [34]:
x_train_rf3 = []
y_train_rf3 = []
for i in range(len(xtrain)):
    x_train_rf3.append([predict_y_cnn[i] , predict_y_lstm[i] , predict_y_gru[i] , predict_y_dt[i] , predict_y_rf1[i] , predict_y_knn[i] , predict_y_lr[i] , predict_y_svm[i] , predict_y_rf2[i]])
    y_train_rf3.append(ytrain.iloc[i])


In [35]:
y_train_rf3 = np.array(y_train_rf3)


In [36]:
x_train_rf3

[[1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 0, 1, 1, 1, 1, 1],
 [1, 1, 1, 0, 1, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 1],
 [0, 0, 0, 0, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 1, 0, 0, 1],
 [1, 1, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0],
 [1, 1, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [1, 1, 0, 1, 1, 0, 1, 1, 1],
 [0, 0, 0, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 1, 0, 0, 0],
 [0, 0, 0, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 0, 1, 1, 1, 1, 1],
 [0, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 1, 1, 1, 1, 1, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 0,

In [37]:
y_train_rf3

array([1, 1, 1, ..., 0, 1, 1], dtype=int64)

In [38]:
rf3_model = RandomForestClassifier(n_estimators=1000, random_state=42)

rf3_model.fit(x_train_rf3, y_train_rf3)


RandomForestClassifier(n_estimators=1000, random_state=42)

In [50]:
pickle.dump(rf3_model, file = open("rf3_model_kdnugget.pickle", "wb"))


### Test

In [39]:
predict_y_lr_test = log_model.predict(kdnugget_X_test)



In [40]:
predict_y_svm_test = svm_clf.predict(kdnugget_X_test)


In [41]:
predict_y_dt_test = dt_clf.predict(kdnugget_X_test)

In [42]:
predict_y_knn_test = knn.predict(kdnugget_X_test)

In [43]:
predict_y_cnn_test = cnn_model.predict(xtest_pad).round().astype(int).reshape(-1)

29/29 [==============================] - 0s 5ms/step


In [44]:
predict_y_gru_test = gru_model.predict(xtest_pad).round().astype(int).reshape(-1)

29/29 [==============================] - 1s 27ms/step


In [45]:
predict_y_lstm_test = lstm_model.predict(xtest_pad).round().astype(int).reshape(-1)

29/29 [==============================] - 2s 57ms/step


In [46]:
predict_y_rf2_test = rf2_clf.predict(kdnugget_X_test_tf)

In [47]:
predict_y_rf1_test = rf1_clf.predict(kdnugget_X_test)

In [48]:
x_test_rf3 = []
y_test_rf3 = []
for i in range(len(xtest)):
    x_test_rf3.append([predict_y_cnn_test[i] , predict_y_lstm_test[i] , predict_y_gru_test[i] , predict_y_dt_test[i] , predict_y_rf1_test[i] , predict_y_knn_test[i] , predict_y_lr_test[i] , predict_y_svm_test[i] , predict_y_rf2_test[i]])
    y_test_rf3.append(ytest.iloc[i])

y_test_rf3 = np.array(y_test_rf3)


In [51]:
# rf3_model = pickle.load(open("rf3_model_kdnugget.pickle", "rb"))
predict_y_rf3 = rf3_model.predict(x_test_rf3)

In [52]:
eval_model(y_test_rf3 , predict_y_rf3)

accuracy:   0.9902067464635473
Precision:   0.9911504424778761
Recall:   0.9889624724061811
F1 score:   0.9900552486187846
